# Coursework 1: ML basics and fully-connected networks

#### Instructions

Please submit a version of this notebook containing your answers on CATe as *CW1*. Write your answers in the cells below each question.

We recommend that you work on the Ubuntu workstations in the lab. This assignment and all code were only tested to work on these machines. In particular, we cannot guarantee compatibility with Windows machines and cannot promise support if you choose to work on a Windows machine.

You can work from home and use the lab workstations via ssh (for list of machines: https://www.doc.ic.ac.uk/csg/facilities/lab/workstations). 

Once logged in, run the following commands in the terminal to set up a Python environment with all the packages you will need.

    export PYTHONUSERBASE=/vol/bitbucket/nuric/pypi
    export PATH=/vol/bitbucket/nuric/pypi/bin:$PATH

Add the above lines to your `.bashrc` to have these enviroment variables set automatically each time you open your bash terminal.

Any code that you submit will be expected to run in this environment. Marks will be deducted for code that fails to run.

Run `jupyter-notebook` in the coursework directory to launch Jupyter notebook in your default browser.

DO NOT attempt to create a virtualenv in your home folder as you will likely exceed your file quota.

**DEADLINE: 7pm, Tuesday 5th February, 2019**

## Part 1

1. Describe two practical methods used to estimate a supervised learning model's performance on unseen data. Which strategy is most commonly used in most deep learning applications, and why?
2. Suppose that you have reason to believe that your multi-layer fully-connected neural network is overfitting. List four things that you could try to improve generalization performance.

\**ANSWERS FOR PART 1 IN THIS CELL*\*
1. Cross-validation, percentage split. Best option for deep learning applications --> Percentage split?
2. Add more training data, Change Regularisation, Change features, Change flexibility (reduce degree of polynomial) 

## Part 2

1. Why can gradient-based learning be difficult when using the sigmoid or hyperbolic tangent functions as hidden unit activation functions in deep, fully-connected neural networks?
2. Why is the issue that arises in the previous question less of an issue when using such functions as output unit activation functions, provided that an appropriate loss function is used?
3. What would happen if you initialize all the weights to zero in a multi-layer fully-connected neural network and attempt to train your model using gradient descent? What would happen if you did the same thing for a logistic regression model?

\**ANSWERS FOR PART 2 IN THIS CELL*\*
1. With gradient-based learning, when calcuting the gradient and the sigmoid functions return values really close to 0, the weights don't update as the gradient is very close to 0. (vanishing gradient) Saturation of the sigmoid prevents gradent based learning from making good progress. The sigmoid saturates to 0 when z becomes very negative and saturates to 1 when z becomes very positive. The gradient can shrink too small to be suseful.
2. When they are used as output unit activation functions, an appropriate cost function is necessary to undo the saturation.
3. 
* For neural networks the cost function does not have just one optimal pointand tend to get stuck in local minima, so it's a good idea to give them many different starting values. You can't do that if they all start at zero.
* If the neurons start with the same weights, then all the neurons will follow the same gradient, and will always end up doing the same thing as one another.
* Whenever you have a convex cost function you are allowed to initialize your weights to zeros. The cost function of logistic regression is convex and so no matter what the weights are initialised at, grade


## Part 3

In this part, you will use PyTorch to implement and train a multinomial logistic regression model to classify MNIST digits.

Restrictions:
* You must use (but not modify) the code provided in `utils.py`. **This file is deliberately not documented**; read it carefully as you will need to understand what it does to complete the tasks.
* You are NOT allowed to use the `torch.nn` module.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultinomialLogisticRegressionClassifier` class below by filling in the missing parts (expected behaviour is prescribed in the documentation):
    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 92% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.


In [1]:
from utils import *
import numpy as np

In [2]:
# *CODE FOR PART 3.1 IN THIS CELL*

class MultinomialLogisticRegressionClassifier:
    def __init__(self, weight_init_sd=100.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        self.weight_init_sd = weight_init_sd

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        self.weights = torch.randn((784,10)) * self.weight_init_sd
        self.weights.requires_grad_()
        self.bias = torch.zeros(10)
        self.bias.requires_grad_()
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)
    
    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be a Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        inputs_vec = inputs.view(-1, 784)
     
        y = torch.mm(inputs_vec, self.weights) + self.bias
        
#         return F.log_softmax(z, dim=1)
        return y - torch.log(torch.sum(torch.exp(y), 1)).unsqueeze(1)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return [self.weights, self.bias]
#         return [{"params": self.weights}, {"params": self.bias}]
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return torch.sum(torch.abs(self.weights))
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return torch.sum(self.weights ** 2)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [3]:
# *CODE FOR PART 3.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *
torch.manual_seed(0)
model = MultinomialLogisticRegressionClassifier(weight_init_sd=0.0001)
res = run_experiment(
    model,
    optimizer=optim.Adam(model.parameters(), 0.001),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=0.000001,
    l2_penalty_coef=0.000001,
    suppress_output=False
)

Epoch 0: training...
Train set:	Average loss: 0.5640, Accuracy: 0.8684
Validation set:	Average loss: 0.3583, Accuracy: 0.9018

Epoch 1: training...
Train set:	Average loss: 0.3301, Accuracy: 0.9093
Validation set:	Average loss: 0.3085, Accuracy: 0.9123

Epoch 2: training...
Train set:	Average loss: 0.3001, Accuracy: 0.9164
Validation set:	Average loss: 0.2908, Accuracy: 0.9163

Epoch 3: training...
Train set:	Average loss: 0.2860, Accuracy: 0.9203
Validation set:	Average loss: 0.2819, Accuracy: 0.9192

Epoch 4: training...
Train set:	Average loss: 0.2771, Accuracy: 0.9232
Validation set:	Average loss: 0.2787, Accuracy: 0.9198

Epoch 5: training...
Train set:	Average loss: 0.2710, Accuracy: 0.9244
Validation set:	Average loss: 0.2756, Accuracy: 0.9212

Epoch 6: training...
Train set:	Average loss: 0.2662, Accuracy: 0.9259
Validation set:	Average loss: 0.2725, Accuracy: 0.9215

Epoch 7: training...
Train set:	Average loss: 0.2626, Accuracy: 0.9270
Validation set:	Average loss: 0.2684, Ac

## Part 4

In this part, you will use PyTorch to implement and train a multi-layer fully-connected neural network to classify MNIST digits.

Your network must have three hidden layers with 128, 64, and 32 hidden units respectively.

The same restrictions as in Part 3 apply.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultilayerClassifier` class below by filling in the missing parts of the following methods (expected behaviour is prescribed in the documentation):

    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 97% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.

3. Describe an alternative strategy for initializing weights that may perform better than the strategy we have used here.

In [4]:
# *CODE FOR PART 4.1 IN THIS CELL*

class MultilayerClassifier:
    def __init__(self, activation_fun="sigmoid", weight_init_sd=1.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        super().__init__()
        self.activation_fun = activation_fun
        self.weight_init_sd = weight_init_sd

        if self.activation_fun == "relu":
            self.activation = F.relu
        elif self.activation_fun == "sigmoid":
            self.activation = torch.sigmoid
        elif self.activation_fun == "tanh":
            self.activation = torch.tanh
        else:
            raise NotImplementedError()

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        
        # 128, 64, and 32 hidden units
        self.w0 = torch.randn((784,128)) * self.weight_init_sd
        self.w0.requires_grad_()
        self.b0 = torch.zeros(128)
        self.b0.requires_grad_()
        
        self.w1 = torch.randn((128,64)) * self.weight_init_sd
        self.w1.requires_grad_()
        self.b1 = torch.zeros(64)
        self.b1.requires_grad_()
        
        self.w2 = torch.randn((64,32)) * self.weight_init_sd
        self.w2.requires_grad_()
        self.b2 = torch.zeros(32)
        self.b2.requires_grad_()
        
        self.w3 = torch.randn((32,10)) * self.weight_init_sd
        self.w3.requires_grad_()
        self.b3 = torch.zeros(10)
        self.b3.requires_grad_()
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        inputs_vec = inputs.view(-1, 784)

        y0 = torch.mm(inputs_vec, self.w0) + self.b0
        
        a1 = self.activation(y0)
        y1 = torch.mm(a1, self.w1) + self.b1
        
        a2 = self.activation(y1)
        y2 = torch.mm(a2, self.w2) + self.b2
        
        a3 = self.activation(y2)
        y3 = torch.mm(a3, self.w3) + self.b3
        
        return y3 - torch.log(torch.sum(torch.exp(y3), 1)).unsqueeze(1)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
#         return [{"params": self.w0}, {"params": self.b0}, {"params": self.w1}, {"params": self.b1}, \
#                 {"params": self.w2}, {"params": self.b2}, {"params": self.w3}, {"params": self.b3}]
        return [self.w0, self.b0, self.w1, self.b1, self.w2, self.b2, self.w3, self.b3]
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return torch.sum(torch.abs(self.w0)) + torch.sum(torch.abs(self.w1)) + \
                 torch.sum(torch.abs(self.w2)) + torch.sum(torch.abs(self.w3))
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. sum of squared values 
        of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return torch.sum(self.w0 ** 2) + torch.sum(self.w1 ** 2) + \
                torch.sum(self.w2 ** 2) + torch.sum(self.w3 ** 2)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [5]:
# *CODE FOR PART 4.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *
torch.manual_seed(0)
model = MultilayerClassifier(activation_fun='tanh', weight_init_sd=0.0001)
res = run_experiment(
    model,
    optimizer=optim.Adam(model.parameters(), 0.002),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=9,
    l1_penalty_coef=0.0000001,
    l2_penalty_coef=0.0000001,
    suppress_output=False
)

Epoch 0: training...
Train set:	Average loss: 1.2335, Accuracy: 0.5263
Validation set:	Average loss: 0.3156, Accuracy: 0.9218

Epoch 1: training...
Train set:	Average loss: 0.2292, Accuracy: 0.9372
Validation set:	Average loss: 0.1690, Accuracy: 0.9527

Epoch 2: training...
Train set:	Average loss: 0.1373, Accuracy: 0.9607
Validation set:	Average loss: 0.1310, Accuracy: 0.9625

Epoch 3: training...
Train set:	Average loss: 0.1006, Accuracy: 0.9705
Validation set:	Average loss: 0.1226, Accuracy: 0.9647

Epoch 4: training...
Train set:	Average loss: 0.0829, Accuracy: 0.9753
Validation set:	Average loss: 0.1135, Accuracy: 0.9672

Epoch 5: training...
Train set:	Average loss: 0.0636, Accuracy: 0.9813
Validation set:	Average loss: 0.1026, Accuracy: 0.9705

Epoch 6: training...
Train set:	Average loss: 0.0548, Accuracy: 0.9831
Validation set:	Average loss: 0.1150, Accuracy: 0.9665

Epoch 7: training...
Train set:	Average loss: 0.0483, Accuracy: 0.9849
Validation set:	Average loss: 0.1121, Ac

\**ANSWERS FOR PART 4.3 IN THIS CELL*\*